In [ ]:
!pip install kaggle

In [ ]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory


In [ ]:
#Using api to import data through kaggle

!kaggle datasets download -d kazanova/sentiment140


Dataset URL: https://www.kaggle.com/datasets/kazanova/sentiment140
License(s): other
 87% 70.0M/80.9M [00:00<00:00, 158MB/s]
100% 80.9M/80.9M [00:00<00:00, 147MB/s]


In [ ]:
import zipfile
import pandas as pd
import numpy as np
import re
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score



In [ ]:
from zipfile import  ZipFile
data = '/content/sentiment140.zip'

with ZipFile(data,'r') as zip:
  zip.extractall()
  print('The dataset has been extracted')


The dataset has been extracted


In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('wordnet')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...


True

In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv', encoding='ISO 8859 1')

In [ ]:
twitter_data.shape

(1599999, 6)

In [ ]:
twitter_data.head()

,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, that's a bummer. You shoulda got David Carr of Third Day to do it. ;D"
0,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
1,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
2,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
3,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."
4,0,1467811372,Mon Apr 06 22:20:00 PDT 2009,NO_QUERY,joy_wolf,@Kwesidei not the whole crew


In [ ]:
#resetting the data

col_name = ['target', 'ids','date', 'flag', 'user', 'text']


In [ ]:
twitter_data = pd.read_csv('/content/training.1600000.processed.noemoticon.csv',names=col_name, encoding='ISO 8859 1')

In [ ]:
twitter_data.head()

,target,ids,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


0 - Negative, 4-Positive


In [ ]:
twitter_data.isnull().sum()

,0
target,0
ids,0
date,0
flag,0
user,0
text,0


In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
4,800000


Converting the target '4' to '1'

In [ ]:
twitter_data.replace({'target':{4:1}}, inplace=True)

In [ ]:
twitter_data['target'].value_counts()

,count
target,
0,800000
1,800000


In [ ]:
stem = PorterStemmer()
lem  = WordNetLemmatizer()

In [ ]:
import re
from nltk.stem import PorterStemmer
from nltk.corpus import stopwords

# Initialize the stemmer
stem = PorterStemmer()

def stemming(content):
    # Remove non-alphabetical characters and convert to lowercase
    content = re.sub('[^A-Za-z]', ' ', content)
    content = content.lower()

    # Split the content into words
    words = content.split()

    # Remove stopwords and apply stemming
    stemmed = [stem.stem(word) for word in words if word not in set(stopwords.words('english'))]

    # Join the stemmed words back into a single string
    stemmed = ' '.join(stemmed)

    return stemmed


In [ ]:
twitter_data['stemmed_con'] = twitter_data['text'].apply(stemming)


In [ ]:
twitter_data

,target,ids,date,flag,user,text,stemmed_con
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t...",switchfoot http twitpic com zl awww bummer sho...
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...,upset updat facebook text might cri result sch...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...,kenichan dive mani time ball manag save rest g...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire,whole bodi feel itchi like fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all....",nationwideclass behav mad see
...,...,...,...,...,...,...,...
1599995,1,2193601966,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,AmandaMarie1028,Just woke up. Having no school is the best fee...,woke school best feel ever
1599996,1,2193601969,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,TheWDBoards,TheWDB.com - Very cool to hear old Walt interv...,thewdb com cool hear old walt interview http b...
1599997,1,2193601991,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,bpbabe,Are you ready for your MoJo Makeover? Ask me f...,readi mojo makeov ask detail
1599998,1,2193602064,Tue Jun 16 08:40:49 PDT 2009,NO_QUERY,tinydiamondz,Happy 38th Birthday to my boo of alll time!!! ...,happi th birthday boo alll time tupac amaru sh...


In [ ]:
print(twitter_data['stemmed_con'])

0          switchfoot http twitpic com zl awww bummer sho...
1          upset updat facebook text might cri result sch...
2          kenichan dive mani time ball manag save rest g...
3                            whole bodi feel itchi like fire
4                              nationwideclass behav mad see
                                 ...                        
1599995                           woke school best feel ever
1599996    thewdb com cool hear old walt interview http b...
1599997                         readi mojo makeov ask detail
1599998    happi th birthday boo alll time tupac amaru sh...
1599999    happi charitytuesday thenspcc sparkschar speak...
Name: stemmed_con, Length: 1600000, dtype: object


In [ ]:
X = twitter_data['stemmed_con'].values
y = twitter_data['target'].values

In [ ]:
X

array(['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
       'upset updat facebook text might cri result school today also blah',
       'kenichan dive mani time ball manag save rest go bound', ...,
       'readi mojo makeov ask detail',
       'happi th birthday boo alll time tupac amaru shakur',
       'happi charitytuesday thenspcc sparkschar speakinguph h'],
      dtype=object)

In [ ]:
X_train , X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)

In [ ]:
print(X_train.shape, X_test.shape)

(1280000,) (320000,)


In [ ]:
#converting words to vectors


tf = TfidfVectorizer()

In [ ]:
X_train = tf.fit_transform(X_train)
X_test = tf.transform(X_test)

In [ ]:
model = LogisticRegression(max_iter=1000)

In [ ]:
model.fit(X_train,y_train)

LogisticRegression(max_iter=1000)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
print(accuracy_score(y_test,y_pred))

0.775465625


NLP using **DL**

In [ ]:
import tensorflow as tf

from tensorflow.keras.layers import Embedding
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [ ]:
message = twitter_data['stemmed_con'].copy()

In [ ]:
message.tolist()

['switchfoot http twitpic com zl awww bummer shoulda got david carr third day',
 'upset updat facebook text might cri result school today also blah',
 'kenichan dive mani time ball manag save rest go bound',
 'whole bodi feel itchi like fire',
 'nationwideclass behav mad see',
 'kwesidei whole crew',
 'need hug',
 'loltrish hey long time see ye rain bit bit lol fine thank',
 'tatiana k nope',
 'twittera que muera',
 'spring break plain citi snow',
 'pierc ear',
 'caregiv bear watch thought ua loss embarrass',
 'octolinz count idk either never talk anymor',
 'smarrison would first gun realli though zac snyder doucheclown',
 'iamjazzyfizzl wish got watch miss iamlilnicki premier',
 'holli death scene hurt sever watch film wri director cut',
 'file tax',
 'lettya ahh ive alway want see rent love soundtrack',
 'fakerpattypattz oh dear drink forgotten tabl drink',
 'alydesign day get much done',
 'one friend call ask meet mid valley today time sigh',
 'angri barista bake cake ate',
 'week g

In [ ]:
vocab = 10000
oneH = [one_hot(words,vocab)for words in message]

In [ ]:
message[0]

'switchfoot http twitpic com zl awww bummer shoulda got david carr third day'

In [ ]:
oneH[0]

[7876, 9582, 7328, 8702, 6169, 4848, 6173, 425, 76, 6767, 4206, 5423, 2487]

In [ ]:
sent = 25
embedded = pad_sequences(oneH, padding='pre', maxlen=sent)

In [ ]:
embedded[0]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0, 7876, 9582, 7328, 8702, 6169, 4848, 6173,  425,   76, 6767,
       4206, 5423, 2487], dtype=int32)

In [ ]:
dim = 15

In [ ]:
modell = Sequential()
modell.add(Embedding(input_dim=vocab, output_dim=dim, input_length=sent ))
modell.add(Dropout(0.3))
modell.add(LSTM(100))
modell.add(Dropout(0.3))
modell.add(Dense(1, activation='sigmoid'))
modell.compile(loss='binary_crossentropy', optimizer='Adam', metrics=['accuracy'])
modell.build(input_shape=(None, sent))
modell.summary()

/usr/local/lib/python3.11/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(


Model: "sequential_2"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                         ┃ Output Shape                ┃         Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━┩
│ embedding_2 (Embedding)              │ (None, 25, 15)              │         150,000 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_4 (Dropout)                  │ (None, 25, 15)              │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ lstm_2 (LSTM)                        │ (None, 100)                 │          46,400 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dropout_5 (Dropout)                  │ (None, 100)                 │               0 │
├──────────────────────────────────────┼─────────────────────────────┼─────────────────┤
│ dense_2 (Dense)                      │ (None, 1)                   │             101 │
└──────────────────────────────────────┴─────────────────────────────┴─────────────────┘

 Total params: 196,501 (767.58 KB)

 Trainable params: 196,501 (767.58 KB)

 Non-trainable params: 0 (0.00 B)

In [ ]:
len(embedded), y.shape

(1600000, (1600000,))

In [ ]:
X_final = np.array(embedded)
y_final = np.array(y)

In [ ]:
X_final

array([[   0,    0,    0, ..., 4206, 5423, 2487],
       [   0,    0,    0, ..., 9772, 3992, 5285],
       [   0,    0,    0, ..., 3545, 6292, 8569],
       ...,
       [   0,    0,    0, ..., 1062, 5344, 3311],
       [   0,    0,    0, ..., 4545,  974, 2854],
       [   0,    0,    0, ..., 3729, 8593, 2469]], dtype=int32)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_final, y_final, test_size=0.3, random_state=13)

In [ ]:
modell.fit(X_train, y_train, validation_data=(X_test,y_test), batch_size=5000, epochs=10)

Epoch 1/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 316s 1s/step - accuracy: 0.7474 - loss: 0.5146 - val_accuracy: 0.7536 - val_loss: 0.5031
Epoch 2/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 350s 2s/step - accuracy: 0.7531 - loss: 0.5044 - val_accuracy: 0.7556 - val_loss: 0.4990
Epoch 3/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 334s 1s/step - accuracy: 0.7560 - loss: 0.4996 - val_accuracy: 0.7569 - val_loss: 0.4967
Epoch 4/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 342s 2s/step - accuracy: 0.7564 - loss: 0.4981 - val_accuracy: 0.7573 - val_loss: 0.4957
Epoch 5/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 382s 2s/step - accuracy: 0.7585 - loss: 0.4955 - val_accuracy: 0.7584 - val_loss: 0.4946
Epoch 6/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 341s 1s/step - accuracy: 0.7591 - loss: 0.4932 - val_accuracy: 0.7588 - val_loss: 0.4935
Epoch 7/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 342s 2s/step - accuracy: 0.7590 - loss: 0.4920 - val_accuracy: 0.7592 - val_loss: 0.4928
Epoch 8/10
224/224 ━━━━━━━━━━━━━━━━━━━━ 383s 2s/step - accuracy: 0.7603 - loss: 0.4902 - val_accu

In [ ]:
y_predd = modell.predict(X_test)

15000/15000 ━━━━━━━━━━━━━━━━━━━━ 118s 8ms/step


In [ ]:
y_predd = np.where(y_predd>0.5, 1, 0)

In [ ]:
from sklearn.metrics import accuracy_score,classification_report,confusion_matrix

In [ ]:
print(accuracy_score(y_test, y_predd))
print('-------------------------------------')
print(classification_report(y_test,y_predd))
print('------------------------------------------')
print(confusion_matrix(y_test,y_predd))
print('-------------------------------')

0.7613375
-------------------------------------
              precision    recall  f1-score   support

           0       0.76      0.76      0.76    240296
           1       0.76      0.76      0.76    239704

    accuracy                           0.76    480000
   macro avg       0.76      0.76      0.76    480000
weighted avg       0.76      0.76      0.76    480000

------------------------------------------
[[183152  57144]
 [ 57414 182290]]
-------------------------------


In [ ]:
import pickle

In [ ]:
filename = 'sentiment.sav'
pickle.dump(modell, open(filename, 'wb'))